In [1]:
import itertools
import pandas as pd
import cobra.flux_analysis
from cobra import Metabolite, Reaction, Model
import time
import numpy as np
from functools import partial
from src.mp_functions import combinations_subset, parallelize_dataframe, knockout_FBA, knockout_FBA_w_tasks

from functools import partial
from src.met_task_functions import constrain_model, read_tasks


"""A mess of a document with different code cells.
Good to to use for any testing that involves the Recon3D model as it takes some time to load in."""

start_time = time.time()
model_file_path = 'C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/GTEx/brain.xml'
model = cobra.io.read_sbml_model(model_file_path)

model_list = constrain_model(model, ALLMETSIN=True)
#Removes unused model.
model_list[2] = None
end_time = time.time()
print('Model load and preparation time: %.6f seconds' % (end_time - start_time))

Academic license - for non-commercial use only - expires 2022-10-03
Using license file c:\gurobi\gurobi.lic
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmprygh75nc.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmp8mstm15f.lp
Reading time = 0.07 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Read LP format model from file C:\Users\Sigve\AppData\Local\Temp\tmpki502gue.lp
Reading time = 0.06 seconds
: 5482 rows, 15138 columns, 66608 nonzeros
Model load and preparation time: 73.868981 seconds


In [22]:
def tasks_test(task_list: list, model_list: list, gene_ids: list) -> list:
    """Performs knockout FBA and checks tasks for the knockout. Returns a list of the results; objective value for
    general FBA and pass/fail for the tasks."""
    with model_list[0]:
        for gene_id in gene_ids:
            try:
                model_list[0].genes.get_by_id(gene_id).knock_out()
            except KeyError:
                return gene_id + ' not in model.'
        res = [model_list[0].slim_optimize()]

    for task in task_list:
        t_model = model_list[task[3]]

        with t_model:
            for subset in [task[0], task[1]]:
                for rx in subset:
                    if rx == 'ALLMETSIN':
                        # Adds boundary metabolites for other reactions when ALLMETSIN is used
                        for r in subset[1:]:
                            for m2 in r.metabolites:
                                for r2 in m2.reactions:
                                    if r2.boundary and r2.id != r.id:
                                        r2.add_metabolites({Metabolite(
                                                            m2.id[:-4] + 'x[x]',
                                                            formula=m2.formula,
                                                            name=' '.join(m2.name.split(' ')[:-1]) + ' [Boundary]',
                                                            compartment='x'): 1})
                        continue
                    t_model.add_reaction(rx)

            if task[2] != 'nan':
                t_model.add_reaction(task[2])

            for gene_id in gene_ids:
                t_model.genes.get_by_id(gene_id).knock_out()

            if t_model.slim_optimize(error_value='nan') == 'nan':
                res += [0]
            else:
                res += [1]


    return res

In [2]:
# Read and format task data
task_list = read_tasks('C:/Users/Sigve/Genome_Data/Human1/Human1_GEM/tasks/essential_tasks.tsv', model_list)

In [3]:
# Read test data
test_data = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/test_data.csv')
test_data['pass/fail'] = test_data.values[:, 4:].tolist()
test_data = test_data[['phewas_code', 'gene_ids', 'solution', 'pass/fail']]
test_data['gene_ids'] = test_data['gene_ids'].apply(lambda x: x.split(','))
print(test_data.head())

# Reduce number of entries
#test_data = test_data.iloc[:10, :]

   phewas_code                            gene_ids   solution  \
0       290.11  [ENSG00000100030, ENSG00000197375] -80.729783   
1       290.11  [ENSG00000101577, ENSG00000197375] -80.729783   
2       290.11  [ENSG00000102780, ENSG00000197375] -80.729783   
3       290.11  [ENSG00000107798, ENSG00000197375] -80.729783   
4       290.11  [ENSG00000113448, ENSG00000197375] -80.729783   

                                           pass/fail  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


In [18]:
g = ['ENSG00000100030', 'ENSG00000197375']
%load_ext line_profiler
%lprun -f tasks_test tasks_test(task_list, model_list, g)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
*** KeyboardInterrupt exception caught in code being profiled.

In [25]:
gene_list = [[[g.id]] for g in model.genes]
print(len(gene_list))
gene_list = pd.DataFrame(gene_list, columns=['gene_ids'])
print(gene_list.head())

2451
            gene_ids
0  [ENSG00000000419]
1  [ENSG00000000938]
2  [ENSG00000001036]
3  [ENSG00000001084]
4  [ENSG00000001630]


In [39]:
start_time = time.time()

#test_data['results'] = test_data['gene_ids'].apply(partial(tasks_test, task_list, model_list))
test_results= parallelize_dataframe(gene_list, partial(combinations_subset, partial(knockout_FBA_w_tasks, task_list, model_list)), 16)


end_time = time.time()
print('FBA runtime: %.6f seconds' % (end_time - start_time))

FBA runtime: 5325.673097 seconds


In [40]:
copy = test_results.copy()

In [41]:
test_results['solution'] = test_results['results'].apply(lambda x: x[0])
test_results['results'] = test_results['results'].apply(lambda x: x[1:])
test_results['tasks_results'] = test_results['results']
test_results = test_results[['gene_ids', 'solution', 'tasks_results']]

In [42]:
test_results.to_csv('C:/Users/Sigve/Genome_Data/results/model_tests/brain_single_del.tsv', sep='\t')

In [43]:
print(len(test_results['tasks_results'].iloc[0]))

57


In [44]:
print(test_results.head())
for d in test_results['results'].head():
    print(d)

            gene_ids   solution  \
0  [ENSG00000000419]  87.693483   
1  [ENSG00000000938]  87.693483   
2  [ENSG00000001036]  87.693483   
3  [ENSG00000001084]  87.693483   
4  [ENSG00000001630]   0.000000   

                                       tasks_results  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


KeyError: 'results'

In [64]:
essential = pd.read_csv('C:/Users/Sigve/Genome_Data/results/model_tests/brain_model_essential.csv', index_col=0)

essential['pass/fail'] = essential.values.tolist()
essential = essential[['pass/fail']].reset_index()
print(essential.shape[0])
essential['pass/fail'] = essential['pass/fail'].apply(lambda x: x[:-1] + [int(x[-1][0])])

#essential = essential[essential['pass/fail'].map(lambda x: any(task == 1 for task in x))]
essential['gene_ids'] = essential['Var1'].apply(lambda x: [x])
essential = pd.DataFrame(essential[['gene_ids', 'pass/fail']])
essential['pass/fail'] = essential['pass/fail'].apply(lambda x: [1 if i==0 else 0 for i in x])
essential.reset_index(inplace=True, drop=True)
print(essential.head())
print(essential.shape[0])

2451
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [65]:
comparison_df = test_results.copy()
comparison_df['MATLAB'] = essential['pass/fail']
print(comparison_df.head())

            gene_ids   solution  \
0  [ENSG00000000419]  87.693483   
1  [ENSG00000000938]  87.693483   
2  [ENSG00000001036]  87.693483   
3  [ENSG00000001084]  87.693483   
4  [ENSG00000001630]   0.000000   

                                       tasks_results  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                              MATLAB  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


In [54]:
#comparison_df['MATLAB'] = comparison_df['MATLAB'].apply(lambda x: x[:-1] + [abs(x[-1]-1)])

#print(len(comparison_df['MATLAB'].iloc[0]))

57


In [66]:
# Data comparison

comparison_df['comparison'] = comparison_df[['tasks_results', 'MATLAB']].apply(lambda x: any([False if i == j else True for i, j in zip(x[0], x[1])]), axis=1)
error_df = comparison_df[comparison_df['comparison']]

print('Number of detected errors: ' + str(error_df.shape[0]))


Number of detected errors: 0


In [83]:
# Check which tasks have failed at least once
indicies = []
for ind, data in comparison_df.iterrows():
    i = 0
    for j in data.tasks_results:
        i += 1
        if not bool(j):
            indicies.append(i)

indicies.sort()
print(set(indicies))

In [90]:
# Extract essential on solution/tasks
essential_genes = comparison_df[comparison_df['solution'].map(lambda x: x!=0)]
print(essential_genes.shape[0])
essential_genes = essential_genes[essential_genes['tasks_results'].map(lambda x: not any([True if i==0 else False for i in x]))]
print(essential_genes.shape[0])

2315
2163


In [110]:
# Extract (non-)essential on both solution and tasks

essential_genes = comparison_df[((comparison_df['solution'] != 0) & (comparison_df['tasks_results'].apply(lambda x: not any([True if i==0 else False for i in x]))))]
essential_genes.reset_index(drop=True, inplace=True)
print(essential_genes.shape[0])

pd.DataFrame(essential_genes['gene_ids'].apply(lambda x: x[0]), columns=['gene_ids']).to_csv(path_or_buf='C:/Users/Sigve/Genome_Data/results/model_tests/model_non_ess_genes_ext.tsv', sep='\t')

2163


In [112]:
# Check if any run with zero as solution passed all tasks
essential_genes = comparison_df[comparison_df['solution'].map(lambda x: x==0)]
print(essential_genes.shape[0])
essential_genes = essential_genes[essential_genes['tasks_results'].map(lambda x: not any([True if i==0 else False for i in x]))]
print(essential_genes.shape[0])

136
0
